## Simulation of one round of blockchain-based federated learning 

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


### Config variables for models and training


In [3]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True


### Load data


In [4]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


Files already downloaded and verified
Files already downloaded and verified


### Create two local models


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)



MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


### Train first local model


In [8]:
model1_name = 'bcfl_model1'

# Training
model1.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model1.parameters(), lr=learning_rate, weight_decay=reg)

train(model1, modelpath, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3019
Epoch [1/1], Step [3/245], Loss: 2.2963
Epoch [1/1], Step [4/245], Loss: 2.2884
Epoch [1/1], Step [5/245], Loss: 2.2762
Epoch [1/1], Step [6/245], Loss: 2.2575
Epoch [1/1], Step [7/245], Loss: 2.2355
Epoch [1/1], Step [8/245], Loss: 2.2271
Epoch [1/1], Step [9/245], Loss: 2.2054
Epoch [1/1], Step [10/245], Loss: 2.1763
Epoch [1/1], Step [11/245], Loss: 2.1626
Epoch [1/1], Step [12/245], Loss: 2.1433
Epoch [1/1], Step [13/245], Loss: 2.1169
Epoch [1/1], Step [14/245], Loss: 2.0500
Epoch [1/1], Step [15/245], Loss: 2.0615
Epoch [1/1], Step [16/245], Loss: 2.0413
Epoch [1/1], Step [17/245], Loss: 2.0203
Epoch [1/1], Step [18/245], Loss: 2.0597
Epoch [1/1], Step [19/245], Loss: 2.0752
Epoch [1/1], Step [20/245], Loss: 2.0442
Epoch [1/1], Step [21/245], Loss: 2.0299
Epoch [1/1], Step [22/245], Loss: 1.9978
Epoch [1/1], Step [23/245], Loss: 2.0546
Epoch [1/1], Step [24/245], Loss: 2.0400
Epoch [1/1], Step [25/245

### Train second local model

In [9]:
model2_name = 'bcfl_model2'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, modelpath, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3002
Epoch [1/1], Step [3/245], Loss: 2.2940
Epoch [1/1], Step [4/245], Loss: 2.2777
Epoch [1/1], Step [5/245], Loss: 2.2647
Epoch [1/1], Step [6/245], Loss: 2.2574
Epoch [1/1], Step [7/245], Loss: 2.2379
Epoch [1/1], Step [8/245], Loss: 2.2019
Epoch [1/1], Step [9/245], Loss: 2.1465
Epoch [1/1], Step [10/245], Loss: 2.1681
Epoch [1/1], Step [11/245], Loss: 2.1650
Epoch [1/1], Step [12/245], Loss: 2.0986
Epoch [1/1], Step [13/245], Loss: 2.0828
Epoch [1/1], Step [14/245], Loss: 2.0830
Epoch [1/1], Step [15/245], Loss: 2.1340
Epoch [1/1], Step [16/245], Loss: 2.1054
Epoch [1/1], Step [17/245], Loss: 2.0922
Epoch [1/1], Step [18/245], Loss: 2.0420
Epoch [1/1], Step [19/245], Loss: 2.0794
Epoch [1/1], Step [20/245], Loss: 2.0314
Epoch [1/1], Step [21/245], Loss: 2.0477
Epoch [1/1], Step [22/245], Loss: 1.9625
Epoch [1/1], Step [23/245], Loss: 1.9378
Epoch [1/1], Step [24/245], Loss: 2.0280
Epoch [1/1], Step [25/245

### Test the trained models

In [10]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model1_name + '.ckpt'))
model1.load_state_dict(model_ckpt)
model = model1
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

model = model2
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 41.8 %
Accuracy of the network on the 1000 test images: 42.5 %



### Forward the two local models to the blockchain


In [12]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_local_model, get_all_models, get_model


print(submit_local_model(model1_name, model1.state_dict()))

print(submit_local_model(model2_name, model2.state_dict()))



{'message': 'Model submitted succesfully'}
{'message': 'Model submitted succesfully'}



### Aggregate the local models on the blockchain
TODO this should be triggered automatically when the total users in the blockchain have triggered their updates


In [ ]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates


### Download the new global model


In [ ]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json

# retrieve future global model and convert it again to pytorch model

# TODO change
model_avg_name = model1_name

model_params = get_model(model_avg_name)

global_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
global_model.to(device)
print(global_model)

load_model_from_json(global_model, model_params)
print(model)
   
print(model1.state_dict()['layers.0.bias'])
print(global_model.state_dict()['layers.0.bias'])
# the global model should be the same as model 1


### Test the new global model